In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%cd "/content/drive/MyDrive/Colab Notebooks"

/content/drive/MyDrive/Colab Notebooks


Import Library

In [ ]:
from preparing import *
from model import Net2
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import dlib
import os
from sklearn.model_selection import GridSearchCV, KFold
import numpy as np
import cv2
from tqdm import tqdm

42437191


Load Data

In [ ]:
# n_classes = 7
# TEST_DIR = 'Data/test'
# test_set = datasets.ImageFolder(root = TEST_DIR, transform=transforms.Compose([
#     transforms.Grayscale(),
#     transforms.ToTensor(),
#     transforms.Normalize((0.457,), (0.287,))]))
# test_loader = DataLoader(test_set,batch_size = 1,shuffle = True, num_workers = 4)


# VAL_DIR = 'Data/val'
# val_set = datasets.ImageFolder(root = VAL_DIR, transform=transforms.Compose([
#     transforms.Grayscale(),
#     transforms.ToTensor(),
#     transforms.Normalize((0.457,), (0.287,))]))
# val_loader = DataLoader(val_set,batch_size = 1,shuffle = True, num_workers=4)
n_classes = 7
TEST_DIR = 'Data/test'
test_loader = ImageGenerator(TEST_DIR, batch_size=1,max_dimension=512)
VAL_DIR = 'Data/val'
val_loader = ImageGenerator(VAL_DIR,batch_size=1,max_dimension=512)

Load models

In [ ]:
face_model = Net2().to('cuda')
face_model.load_state_dict(torch.load('model/checkpoint.pt',map_location=torch.device('cuda'))['state_dict'])
face_model.eval()


Test with Net2 model

In [ ]:
from ignite.engine import Engine, Events, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss

VAL_DIR = 'data/val'
val_set = datasets.ImageFolder(root=VAL_DIR,transform=transforms.Compose([
    transforms.Grayscale(),
    transforms.ToTensor(),
    transforms.Normalize((0.457,), (0.287,))
]))
val_loader = DataLoader(val_set,32,shuffle=True, num_workers=4)

criterion = nn.CrossEntropyLoss()
val_metrics = {
    "accuracy": Accuracy(),
    "loss": Loss(criterion)
}
val_evaluator = create_supervised_evaluator(face_model,metrics=val_metrics, device='cpu')

val_evaluator.run(val_loader)
metrics = val_evaluator.state.metrics
print(f"Test Results - Avg accuracy: {metrics['accuracy']:.2f} Avg loss: {metrics['loss']:.2f}")

Test Results - Avg accuracy: 0.68 Avg loss: 0.97


In [ ]:
mouth_model = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.BatchNorm2d(32),
 
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            # nn.Conv2d(64, , kernel_size=3, stride=1, padding=1),
            # nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.BatchNorm2d(64),
 
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), 
            nn.BatchNorm2d(128),
             
            nn.AdaptiveAvgPool2d((2,2)),

            
            nn.Flatten(), 
            nn.Linear(128*2*2, 100),
            nn.ReLU(),
            nn.Linear(100, n_classes),
            nn.Softmax()).to('cuda')
mouth_model.load_state_dict(torch.load('model/mouth.pth', map_location = 'cuda'))
mouth_model.eval()

In [ ]:
nose_model = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.BatchNorm2d(64),
 
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            # nn.Conv2d(64, , kernel_size=3, stride=1, padding=1),
            # nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.BatchNorm2d(128),
 
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), 
            nn.BatchNorm2d(256),
             
            nn.AdaptiveAvgPool2d((2,2)),

            
            nn.Flatten(), 
            nn.Linear(256*2*2, 128),
            nn.ReLU(),
            nn.Linear(128, n_classes),
            nn.Softmax()).to('cuda')
nose_model.load_state_dict(torch.load('model/nose.pth',map_location='cuda'))
nose_model.eval()

In [ ]:
righteye_model = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.BatchNorm2d(64),
 
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(128,128 , kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.BatchNorm2d(128),
 
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), 
            nn.BatchNorm2d(256),
             
            nn.AdaptiveAvgPool2d((2,2)),

            
            nn.Flatten(), 
            nn.Linear(256*2*2, 128),
            nn.ReLU(),
            nn.Linear(128, n_classes),
            nn.Softmax()).to('cuda')
righteye_model.load_state_dict(torch.load('model/right_eye.pth', map_location='cuda'))
righteye_model.eval()

In [ ]:
lefteye_model = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.BatchNorm2d(64),
 
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128 , kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.BatchNorm2d(128),
 
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), 
            nn.BatchNorm2d(256),
             
            nn.AdaptiveAvgPool2d((2,2)),
            nn.Flatten(), 
            nn.Linear(256*2*2, 200),
            nn.ReLU(),
            nn.Linear(200, n_classes),
            nn.Softmax()).to('cuda')
lefteye_model.load_state_dict(torch.load('model/left_eye.pth', map_location = 'cuda'))
lefteye_model.eval()

Prepare detector and predictor

In [ ]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

In [ ]:
def late_fusion_accuracy(weights, y_pred_A, y_pred_B, y_pred_C, y_pred_D, y_pred_E, y_test):
    y_pred = (y_pred_A + y_pred_B + y_pred_C + y_pred_D + y_pred_E) * weights.unsqueeze(0).expand_as(y_pred_A)
    y_pred_class = torch.argmax(y_pred, dim=1)
    acc = torch.sum(y_pred_class == y_test).float() / y_test.shape[0]
    return acc

In [ ]:
param_grid = [{'weights': [torch.Tensor([0.9,0.5,0.6,0.42,0.45]),
                           torch.Tensor([0.8,0.39,0.3576,0.7,0.3]),
                           torch.Tensor([0.68,0.27,0.2,0.36,0.55])]}]

Set transforms

In [ ]:
tf_roi=transforms.Compose([
transforms.ToPILImage(),
transforms.ToTensor()
])

tf_face = transforms.Compose([
transforms.ToPILImage(),
transforms.Grayscale(),
transforms.ToTensor(),
transforms.Normalize((0.457,), (0.287,))]
)

In [ ]:
predictions_A = []
predictions_B = []
predictions_C = []
predictions_D = []
predictions_E = []
test_labels = []
with torch.no_grad():
    for input, target in tqdm(val_loader):
        target = target.to('cuda')
        test_labels.append(target)
        input = np.einsum('kij->ijk',input.squeeze().numpy()).astype('uint8')
        image = input
        input = tf_face(input).unsqueeze(0)
        input = input.to('cuda')
        output_A = face_model(input.to('cuda'))
        predictions_A.extend(output_A.argmax(dim=1).tolist())  

        # Crop 4 parts of face
        # image = np.einsum('kij->ijk',input.squeeze().numpy()).astype('uint8')
        sub_roi = crop_subregion(image,detector,predictor)
        # Resize if roi's is too small
        output_B = output_C = output_D = output_E = output_A
        for roi in sub_roi:
            img_roi = roi[1]
            if min(img_roi.shape[:-1]) == 0:
                continue
            if min(img_roi.shape[:-1]) < 60:
                img_roi = cv2.resize(img_roi.astype('uint8'),(150,150))
            img_roi = tf_roi(img_roi).unsqueeze(0)
            img_roi = img_roi.to('cuda')
            if roi[0] == 'mouth':
                output_B = mouth_model(img_roi)       
            elif roi[0] == 'nose':  
                output_C = nose_model(img_roi)             
            elif roi[0] == 'right_eye':
                output_D = righteye_model(img_roi)              
            elif roi[0] == 'left_eye':
                output_E = lefteye_model(img_roi)              
        predictions_B.extend(output_B.argmax(dim=1).tolist())
        predictions_C.extend(output_C.argmax(dim=1).tolist())
        predictions_D.extend(output_D.argmax(dim=1).tolist())
        predictions_E.extend(output_E.argmax(dim=1).tolist())
            

  0%|          | 0/3589 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
100%|██████████| 3589/3589 [49:37<00:00,  1.21it/s]


In [ ]:
print(predictions_A)
print(predictions_B)
print(predictions_C)
print(predictions_D)
print(predictions_E)


[0, 5, 5, 0, 0, 4, 0, 0, 0, 0, 2, 4, 5, 0, 0, 5, 0, 4, 0, 0, 0, 0, 0, 0, 0, 5, 2, 0, 0, 4, 0, 0, 0, 0, 0, 0, 5, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 5, 0, 0, 4, 6, 4, 0, 0, 0, 0, 5, 0, 2, 5, 0, 0, 0, 3, 0, 6, 0, 0, 0, 0, 0, 0, 4, 0, 4, 0, 0, 2, 0, 0, 5, 0, 0, 0, 5, 0, 0, 0, 0, 6, 0, 5, 4, 4, 4, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 4, 2, 2, 2, 0, 5, 6, 0, 0, 0, 0, 0, 3, 0, 0, 0, 4, 0, 0, 5, 0, 0, 0, 0, 0, 0, 4, 0, 4, 0, 0, 5, 4, 0, 0, 5, 0, 6, 5, 0, 0, 0, 5, 6, 0, 0, 0, 3, 4, 0, 3, 5, 4, 0, 4, 0, 2, 0, 0, 2, 2, 0, 0, 0, 3, 0, 0, 5, 4, 0, 4, 3, 0, 0, 5, 0, 5, 4, 0, 0, 0, 0, 0, 4, 0, 4, 0, 0, 6, 4, 2, 5, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 4, 3, 0, 0, 5, 0, 5, 4, 0, 0, 0, 2, 4, 0, 2, 0, 3, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 4, 0, 0, 0, 0, 5, 4, 5, 2, 0, 2, 4, 0, 0, 2, 0, 3, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 5, 0, 4, 0, 2, 4, 0, 0, 2, 4, 5, 0, 0, 0, 0, 5, 0, 0, 0, 1, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 4, 0, 

In [ ]:
def late_fusion(models, test_data,weights):
    predictions = []
    for model in models:
        prediction = model(test_data)
        predictions.append(prediction)
    return torch.sum(torch.stack(predictions) * weights, dim = 0)

In [ ]:
models = [face_model, mouth_model, nose_model, righteye_model, lefteye_model]
param_grid = {
    "weights": [torch.tensor([]),
                torch.tensor([]),
                torch.tensor([]),
                torch.tensor([]),
                torch.tensor([])]
}
grid = GridSearchCV(late_fusion,param_grid = param_grid,cv=5)
grid.fit(val_loader)

In [ ]:
# Tính toán độ chính xác bằng late fusion
best_weights = grid_search.best_estimator_.get_params()['weights']
final_prediction = late_fusion_prediction(predictions_A, predictions_B, predictions_C, weights=best_weights)
final_accuracy = accuracy_score(y_test, final_prediction)

NameError: ignored

Test with fusion

In [ ]:
tf_roi=transforms.Compose([
transforms.ToPILImage(),
transforms.ToTensor()
])
tf_face = transforms.Compose([
transforms.ToPILImage(),
transforms.Grayscale(),
transforms.ToTensor(),
transforms.Normalize((0.457,), (0.287,))]
)
def fusion(image,m0,m1,m2,m3,m4):
    # m = [0.67,0.39,0.3576,0.42,0.45]
    sum = m0 + m1 + m2 + m3 + m4
    res_mouth = res_nose = res_righteye = res_lefteye = [0,0,0,0,0,0,0]
    sub_roi = crop_subregion(image,detector,predictor)
    for roi in sub_roi:
        input = roi[1]
        if min(input.shape[:-1]) == 0:
            continue
        if min(input.shape[:-1]) < 60:
            input = cv2.resize(input.astype('uint8'),(150,150))
            # input = input.astype('uint8')
        
        input = tf_roi(input)
        input =  input.unsqueeze(0)
        input = input.to('cuda')
        if roi[0] == 'mouth':
            mouth_out = mouth_model(input).squeeze().tolist()
            res_mouth = list(map(lambda x:x/max(mouth_out)*m1/sum,mouth_out))
        elif roi[0] == 'nose':
            nose_out = nose_model(input).squeeze().tolist()
            res_nose = list(map(lambda x:x/max(nose_out)*m2/sum,nose_out))
        elif roi[0] == 'right_eye':
            righteye_out = righteye_model(input).squeeze().tolist()
            res_righteye = list(map(lambda x:x/max(righteye_out)*m3/sum,righteye_out))
        elif roi[0] == 'left_eye':
            lefteye_out = lefteye_model(input).squeeze().tolist()
            res_lefteye = list(map(lambda x:x/max(lefteye_out)*m4/sum,lefteye_out))
    image = tf_face(image).unsqueeze(0)
    image = image.to('cuda')
    face_out = face_model(image).squeeze().tolist()
    res_face = list(map(lambda x:x/max(face_out)*m0/sum,face_out))
    
    result = []
    for i in range(0,7):
        result.append(res_face[i] + res_mouth[i] + res_nose[i] + res_righteye[i] + res_lefteye[i])
    return result

In [ ]:
a =[1.3,2.3,4.6]
b = [2.5,6.7,3.4]
c = [sum(x) for x in zip(a,b)]

In [ ]:
c

[3.8, 9.0, 8.0]

In [ ]:
loss_fn=nn.CrossEntropyLoss()
accur = 0
total = 0
val_loss = 0
with torch.no_grad():
        for data in tqdm(val_loader):
            image,label=data[0], data[1].to('cuda')
            image = np.einsum('kij->ijk',image.squeeze().numpy()).astype('uint8')
            
            output=torch.tensor(fusion(image,0.9,-0.4,-0.07,-0.1,-0.26)).unsqueeze(0)

            loss= loss_fn(output.to('cuda'),label)
            val_loss+=loss.item()
            
            _, predicted = output.to('cuda').max(1)
            total += label.size(0)
            accur += predicted.eq(label).sum().item()
valll_loss=val_loss/len(test_loader)
accuracy=100.*accur/total

print('Loss: %.3f | Accuracy: %.3f'%(valll_loss,accuracy))

  0%|          | 0/3589 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
100%|██████████| 3589/3589 [15:19<00:00,  3.90it/s]

Loss: 9.318 | Accuracy: 48.677
